In [1]:
# d6598 
# "I bet computers can't make music"
# - Felonious Gru

beat_resolution = 24 # Standard of 24 for full 34 and 44 representation.
                    # Dont go below 8 otherwise risk distortion. 12 is a safe spot to end up
lookback = beat_resolution * 4 * 2  # Approx 2 bars
step = 1  # Skip through it a bit
tempo = 60  # Rough tempo of the samples. Only used for accurate timing of the output
num_pitches = 50  # Must be even cause odd numbers suck
train = False

%matplotlib inline
import os
import functions

Lower bound 39.
Upper bound 89.
Num pitches 50
---
goldberg.mid input shape: (153743, 128)
...Refined down 78 dimensions with 1815 note loss.
...Loss of 0.49 %
...Output shape: (153743, 50)
Extended output shape (153743, 50)


153551 individual phrases.


In [2]:
root = '/Users/James/Projects/PycharmProjects/lstm-generation'
#root = '/home/ubuntu/deep-learning/music/lstm-generation-bach'
#root = '/content....'

midi_dir = os.path.join(root, 'midi')
model_dir = os.path.join(root, 'models')

trainer = functions.Midi(num_pitches, beat_resolution=beat_resolution, midi_dir=midi_dir)
trainer.load_midi(['goldberg.mid'])
trainer.display()

x, y = trainer.vectorise(lookback, step=1)

In [3]:
if train:
    from keras.models import Sequential
    from keras import layers
    from keras.optimizers import RMSprop

    model = Sequential()
    model.add(layers.LSTM(256, input_shape=(lookback, num_pitches), return_sequences=True, dropout=0.1, recurrent_dropout=0.2))
    model.add(layers.LSTM(128, dropout=0.1, recurrent_dropout=0.2, return_sequences=False))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_pitches, activation='sigmoid'))
    model.summary()

    model.compile(loss='categorical_crossentropy',  # categorical_crossentropy or mse
                  optimizer=RMSprop(),
                  metrics=['acc'])

    from time import time
    begin = time()
    for i in range(10):
        print('>Epoch batch', i)
        tic = time()
        history = model.fit(x, y,
                            epochs=10,
                            batch_size=512,
                            verbose=1)
        print('Took %s minutes for epoch. %s minutes elapsed.' % (((time()-tic)/60).__round__(2), ((time()-begin)/60).__round__(2)))
        functions.save_model(model, model_dir, fname='lstm-v13.h5')
    print('Full train took %s minutes.' % ((time()-begin)/60).__round__(2))

else:
    from keras import models
    fname = os.path.join(model_dir, 'lstm-v13.h5')
    model = functions.load_model(fname)

2021-07-03 22:12:43.987272: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded model "/Users/James/Projects/PycharmProjects/lstm-generation/models/lstm-v13.h5"


In [4]:
#temps = [0.5, 0.55, 0.6, 0.7]
temps = [0.6]  # Was best one
outputs = functions.generate_music(model, trainer.roll, lookback, temps, num_pitches, length=lookback*4, noise=True)

functions.multi_save(outputs, 'generated-v13.mid')

Generating with seed index of 25804
Generating roll with temp 0.6 and length 768


2021-07-03 22:12:45.042436: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
/Users/james/Projects/PycharmProjects/lstm-generation/functions.py:150: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


Roll is cut down to only 50 notes
Saving 1 tracks in one file "generated-v13.mid".
Saved file "generated-v13.mid".
